In [1]:
from __future__ import division
import pickle
import os
import medleydb as mdb
import numpy as np
import json
from collections import defaultdict

ImportError: No module named medleydb

In [ ]:
with  open('./medley_artist_index.json') as f:
    artist_label = json.load(f)
    
# with  open('./artist_total_length.pkl') as f:
#    artist_total_length = pickle.load(f)

In [13]:
# get track durations and group by artist

trackList = os.listdir(mdb.AUDIO_PATH)
trackList = [t for t in trackList if t[0]!='.']
mtrack_generator = mdb.load_multitracks(trackList)

track_duration = {}

counter = 1
for track in mtrack_generator:  
    print counter
    track_duration[track.track_id] = track.duration
    counter += 1
    
artist_total_length = defaultdict(int)

for k,v in artist_label.items():
    
    artist_total_length[v]+=track_duration[k]

artist_total_length = zip(artist_total_length.keys(),artist_total_length.values())

with open('./artist_total_length.pkl','w+') as f:
    
    pickle.dump(artist_total_length,f)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [96]:
def generate_datasplit(artist_total_length,artist_label, train = 0.8, validation = 0.1, test = 0.1):
    
    assert train + validation + test == 1

    shuffleOrder = np.random.permutation(artist_total_length)
    l = [float(x[1]) for x in shuffleOrder]
    percent_cum_length = np.cumsum(l)/sum(l)  


    # find split location
    idxTrain = np.abs(percent_cum_length - train).argmin()+1
    idxValidation = np.abs(percent_cum_length - (train+validation)).argmin()+1

    # split data
    train_set = shuffleOrder[:idxTrain]
    validation_set = shuffleOrder[idxTrain:idxValidation]
    test_set = shuffleOrder[idxValidation:]

    # split percentage
    train_length = sum([float(x[1]) for x in train_set])
    validation_length = sum([float(x[1]) for x in validation_set])
    test_length = sum([float(x[1]) for x in test_set])

    total = train_length+validation_length+test_length

    train_percent = train_length/total
    validation_percent = validation_length/total
    test_percent = test_length/total

    train_artist = set([x[0] for x in train_set])
    validation_artist = set([x[0] for x in validation_set])
    test_artist = set([x[0] for x in test_set])
    
    train_track = []
    validation_track = []
    test_track = []

    for k,v in artist_label.items():

        if v in train_artist:

            train_track.append(k)
        elif v in validation_artist:
            validation_track.append(k)
        elif v in test_artist:
            test_track.append(k)
        else:
            print v

    return [train_track, validation_track, test_track]

In [118]:
train = 0.7
validation = 0.1
test = 0.2
nSplits = 20

dataSplits = []
for i in range(nSplits):

    split = generate_datasplit(artist_total_length, artist_label,train = train, validation = validation, test = test)
    dataSplits.append(split)
    

In [105]:

with open('./dataSplits.pkl','w+') as f:
    pickle.dump(dataSplits,f)